<a href="https://colab.research.google.com/github/atakhadivi/Algorithmic-Trading/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q python-binance

In [ ]:
import datetime
from binance.client import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# api key and api secret
api_key = 'ihffj4Q8KBG1NQc8bf6xFjxT9u9b8aZhLXiPFlC0oHXnhzBR6WrnYZwrfkjcoAC0'
api_secret = 'WHcTmbFI5xDCBcXwh9YY5D8UZ0hh4qYYvjuOKcCXfkUr8BAcccBtgRaev2mJTDoV'

In [ ]:
client = Client(api_key, api_secret)
df = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "24 Jun, 2018", "24 Jun, 2020")
df = pd.DataFrame(df,columns= ['Date', 'open', 'high', 'low', 'close', 'volume', 'Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'])
df['Date'] = pd.to_datetime(df['Date'],unit = 'ms')
df.set_index('Date',inplace=True)
df.drop(['Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'],axis=1,inplace=True)
# candles.drop(candles.tail(1).index,inplace=True)
# candles.drop(candles.head(1).index,inplace=True)
# return df
df.to_csv('binance.csv')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", font_scale=1.5)
%matplotlib inline

In [ ]:
df = pd.read_csv("binance.csv", parse_dates=["Date"], index_col="Date")
df.shape
# df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=df.index, y="open", data=df).set_title("Price of BTC")

In [ ]:
df.resample('10D').mean().plot(figsize=(12,6))
plt.figtext(.5,0.9,"Down-sampled to 10-day periods", fontsize=20, ha='center')

In [ ]:
data = df.iloc[:, 0]
hist = []
target = []
length = 700
for i in range(len(data)-length):
    x = data[i:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
print(hist[1][699])


9179.01


In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
target.shape

(32, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
hist_scaled = sc.fit_transform(hist)
target_scaled = sc.fit_transform(target)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

(32, 700, 1)


In [ ]:
X_train = hist_scaled[:30,:,:]
X_test = hist_scaled[30:,:,:]
y_train = target_scaled[:30,:]
y_test = target_scaled[30:,:]

In [ ]:
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
history = model.fit(X_train, y_train, epochs=60, batch_size=32)


In [ ]:
loss = history.history['loss']
epoch_count = range(1, len(loss) + 1)
plt.figure(figsize=(12,8))
plt.plot(epoch_count, loss, 'r--')
plt.legend(['Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('BTC Price Prediction')
plt.legend()
plt.show()

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install seaborn

In [ ]:
!pip install alpha_vantage

In [ ]:
%tensorflow_version 2.x
import json
import requests
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import pandas.util.testing as tm
import matplotlib.pyplot as plt
%matplotlib inline